Instead of doing separate convolution and batch normalization step for each layer, we would like to combine the two steps so that convolution takes care of the normalization. Combine the batch normalization parameters and weights to calculate the batch-normalized weights and biases for only the layer conv_1 of shape 3x3x3x32. You can ignore the biases provided in the model. The formula for calculating BN weights and bias is here: [https://forums.developer.apple.com/thread/65821](https://forums.developer.apple.com/thread/65821). Assume gamma = 1, calculate the weights modifiers and biases for each output channel (32). Do it only for the first convolution layer, conv_1.

BN Weights and Bias Formula
```
A =   g / √(s + 0.001), b = ß - ( A * m )
W = w*A

s: variance
m: mean
g: gamma
ß: beta
w: weights of a feature channel
b: bias of a feature channel
W: batch nomalized weights
for every feature channel separately to get the corresponding weights and bias
```

In [4]:
import os 

import tensorflow as tf

In [9]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.logging.set_verbosity(tf.logging.ERROR)

MODEL = f"../../mobilenet/mobilenet_orig/model.ckpt-906808"
META_GRAPH = f"../../mobilenet/mobilenet_orig/model.ckpt-906808.meta"

In [20]:
tf.reset_default_graph()

In [ ]:
batch_norm/moving_variance:0':

In [32]:
with tf.Session() as sess:
    saver = tf.train.import_meta_graph(META_GRAPH, clear_devices=True)
    saver.restore(sess, MODEL)
    new_weights = {}
    variables = {}
    first_conv_prefix = 'MobileNet/conv_1'
    for variable in tf.global_variables():
        if variable.name.startswith(first_conv_prefix):
            variables[variable.name] = sess.run(variable)
            
    old_weights = variables[f'{first_conv_prefix}/weights:0']
    gamma = 1.0
    beta = variables[f'{first_conv_prefix}/batch_norm/beta:0']
    mean = variables[f'{first_conv_prefix}/batch_norm/moving_mean:0']
    variance = variables[f'{first_conv_prefix}/batch_norm/moving_variance:0']

    # Calculate batch-normalized weights
    A = gamma / tf.math.sqrt(variance + 0.001)
    new_W = old_weights * A
    new_weights[f'{first_conv_prefix}/weights:0'] = new_W

In [35]:
old_weights.shape

(3, 3, 3, 32)

In [36]:
new_weights['MobileNet/conv_1/weights:0'].shape

TensorShape([Dimension(3), Dimension(3), Dimension(3), Dimension(32)])